# basic data processing

In [1]:
import os
import cv2
import glob
import pandas as pd
import numpy as np
import librosa 
import librosa.display
import pylab
import pickle
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential,Model
from tensorflow.keras.layers import concatenate,Activation, Dense, Dropout, Conv2D, Flatten, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling1D, AveragePooling2D, Input, Add, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
from sklearn.metrics import roc_curve
from keras.utils import np_utils
from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

In [11]:
data_dir = '../project/data/Raw_20k/'
file_names = os.listdir('{}'.format(data_dir))
df_list = []

for file_name in file_names:
    with open(data_dir+file_name, 'rb') as f:
        data = pickle.load(f)
    df_list.append(data)
data = pd.concat(df_list)

In [12]:
sort_data =data.sort_values(by=['ProductCount'], axis=0)
sort_data.reset_index(drop=True)

toolcount_list = sort_data['ToolCount'].to_list()
bites = []

bite_number = 1
current_tc = 0
previous_tc = -1

for tc in toolcount_list:
    current_tc = tc

    if current_tc >= previous_tc:
        bites.append(bite_number)

    else:
        bite_number += 1
        bites.append(bite_number)

    previous_tc = current_tc

sort_data['biteCount'] = bites
tmpData = sort_data.reset_index(drop=True)

In [13]:
tmpData.sort_values(by=['biteCount'],ascending=True)

,ProductCount,Current1,Current2,Current3,Vibration1,ToolCount,CurrentTool,Label,biteCount
0,92209,"[0.004208088, 0.004299879, 0.0042128563, 0.004...","[0.0011408329, 0.0011813641, 0.0011920929, 0.0...","[0.0017333031, 0.0018000603, 0.0017404556, 0.0...","[-0.0026130676, -0.0021517277, -0.0015330315, ...",192,1,None,1
80,92289,"[0.005606413, 0.0056409836, 0.0057160854, 0.00...","[0.0011003017, 0.001064539, 0.0010597706, 0.00...","[0.0017845631, 0.0018286705, 0.0017511845, 0.0...","[-0.0027561188, -0.0033974648, -0.0038158894, ...",271,1,None,1
79,92288,"[0.0031113625, 0.0030994415, 0.0029873848, 0.0...","[0.0010538101, 0.0010931492, 0.0012099743, 0.0...","[0.0017940998, 0.00172019, 0.0016880035, 0.001...","[-0.0034987926, -0.0028014183, -0.0023078918, ...",270,1,None,1
78,92287,"[0.003170967, 0.0031423569, 0.0032436848, 0.00...","[0.0011491776, 0.0011754036, 0.0011098385, 0.0...","[0.0022625923, 0.0022637844, 0.002336502, 0.00...","[0.0030696392, 0.004620552, 0.0058448315, 0.00...",269,1,None,1
77,92286,"[0.0035309792, 0.0034761429, 0.0033855438, 0.0...","[0.001270771, 0.0011539459, 0.0011944771, 0.00...","[0.0018954277, 0.0018036366, 0.0017559528, 0.0...","[-0.004144907, -0.0038564205, -0.004440546, -0...",268,1,None,1
...,...,...,...,...,...,...,...,...,...
1788,94108,"[0.0032019615, 0.0030839443, 0.0031507015, 0.0...","[0.0012469292, 0.0011944771, 0.0011610985, 0.0...","[0.0027775764, 0.0028669834, 0.0029051304, 0.0...","[0.001591444, 0.0017738342, 0.0017380714, 0.00...",2,1,None,8
1787,94107,"[0.0031852722, 0.003194809, 0.0031113625, 0.00...","[0.0011146069, 0.0011634827, 0.0012171268, 0.0...","[0.0028717518, 0.0027477741, 0.0027358532, 0.0...","[0.0053215027, 0.0059628487, 0.005978346, 0.00...",1,1,None,8
1814,94134,"[0.0033593178, 0.0034213066, 0.0034046173, 0.0...","[0.001244545, 0.001180172, 0.0011301041, 0.001...","[0.0026202202, 0.0026917458, 0.002721548, 0.00...","[0.00039696693, 0.00081419945, 0.0014281273, 0...",28,1,None,8
1800,94120,"[0.0030946732, 0.0030899048, 0.0031375885, 0.0...","[0.0011467934, 0.0011372566, 0.0011229515, 0.0...","[0.0023078918, 0.002233982, 0.0021839142, 0.00...","[-0.00016331673, -0.00037431717, -0.0003361702...",14,1,None,8


In [14]:
## true False 분리
falseData = pd.DataFrame(columns =['ProductCount','Current1','Current2','Current3','Vibration1','ToolCount','Label'])
trueData = pd.DataFrame(columns = ['ProductCount','Current1','Current2','Current3','Vibration1','ToolCount','Label'])

falseData1 = pd.DataFrame(columns =['ProductCount','Current1','Current2','Current3','Vibration1','ToolCount','Label'])
trueData1 = pd.DataFrame(columns = ['ProductCount','Current1','Current2','Current3','Vibration1','ToolCount','Label'])

for i in range(0,max(tmpData["biteCount"])):
    
    if i == 0 : 
        trueData1 = trueData1.append(tmpData[tmpData["biteCount"] == i+1][-30:],ignore_index = True)
    elif i == 7 :
        falseData1 = falseData1.append(tmpData[tmpData["biteCount"] == i+1][:30],ignore_index= True)
    else :
        falseData = falseData.append(tmpData[tmpData["biteCount"] == i+1][:30],ignore_index= True)
        trueData = trueData.append(tmpData[tmpData["biteCount"] == i+1][-30:],ignore_index = True)
        

falseData['Label'] = False
trueData['Label']=True

falseData1['Label'] = False
trueData1['Label']=True

boundary = round(max(tmpData["biteCount"]) * 0.9)

trainData = (falseData[falseData["biteCount"] < boundary])
trainData =  trainData.append(trueData[trueData["biteCount"] < boundary])

valData = (falseData[falseData["biteCount"] >= boundary])
valData =  valData.append(trueData[trueData["biteCount"] >= boundary])
valData =  valData.append(falseData1)
valData =  valData.append(trueData1)

pickle.dump(trainData, open( "../project/data/experiment/V2/TrainData_XYZC_30.pickle", "wb"))
pickle.dump(valData, open( "../project/data/experiment/V2/ValData_XYZC_30.pickle", "wb"))

<ipython-input-14-b212ac2f46a2>:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trueData1 = trueData1.append(tmpData[tmpData["biteCount"] == i+1][-30:],ignore_index = True)
<ipython-input-14-b212ac2f46a2>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  falseData = falseData.append(tmpData[tmpData["biteCount"] == i+1][:30],ignore_index= True)
<ipython-input-14-b212ac2f46a2>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  trueData = trueData.append(tmpData[tmpData["biteCount"] == i+1][-30:],ignore_index = True)
<ipython-input-14-b212ac2f46a2>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  falseData1 = falseData1.append(tmpData[tm

# for CNN data processing

In [9]:
with open("../project/data/experiment/V2/TrainData_XYZC_20.pickle", 'rb') as f:
    trainData = pickle.load(f)
with open("../project/data/experiment/V2/ValData_XYZC_20.pickle", 'rb') as f:
    valData = pickle.load(f)

In [15]:
def feature_extractor(row):
    name     = str(row[0])
    try:  
        audio1 = np.array(row[1])
        audio2 = np.array(row[2])
        audio3 = np.array(row[3])
        audio4 = np.array(row[4])
        
        # X
        melX = librosa.feature.melspectrogram(y=audio1, sr=20000)
        s_db_X     = librosa.power_to_db(melX, ref=np.max)
        
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_X, sr = 20000, hop_length=64)
        x_path = os.path.join(x_figpath,name+'.png')
        pylab.savefig(x_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # Y
        melY = librosa.feature.melspectrogram(y=audio2, sr=20000)
        s_db_Y     = librosa.power_to_db(melY, ref=np.max)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_Y, sr = 16000, hop_length=64)
        y_path = os.path.join(y_figpath,name+'.png')
        pylab.savefig(y_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # Z
        melZ = librosa.feature.melspectrogram(y=audio3, sr=20000)
        s_db_Z     = librosa.power_to_db(melZ, ref=np.max)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_Z, sr = 20000, hop_length=64)
        z_path = os.path.join(z_figpath,name+'.png')
        pylab.savefig(z_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # C
        melC = librosa.feature.melspectrogram(y=audio4, sr=20000)
        s_db_C     = librosa.power_to_db(melC, ref=np.max)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_C, sr = 20000, hop_length=64)
        c_path = os.path.join(c_figpath,name+'.png')
        pylab.savefig(c_path, bbox_inches=None, pad_inches=0)
        pylab.close()  
        
    except:
        print('file cannot open')
        return None, None, None, None
    return x_path, y_path, z_path, c_path



def feature_extractor_mfcc(row):
    name     = str(row[0])
    try:  
        audio1 = np.array(row[1])
        audio2 = np.array(row[2])
        audio3 = np.array(row[3])
        audio4 = np.array(row[4])
        
        # X
        mfccX = librosa.feature.mfcc(y=audio1, sr=20000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccX, sr = 20000, hop_length=64)
        x_path = os.path.join(x_figpath,name+'.png')
        pylab.savefig(x_path, bbox_inches=None, pad_inches=0)
        pylab.close()     
        
        # Y
        mfccY = librosa.feature.mfcc(y=audio2, sr=20000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccY, sr = 20000, hop_length=64)
        y_path = os.path.join(y_figpath,name+'.png')
        pylab.savefig(y_path, bbox_inches=None, pad_inches=0)
        pylab.close()    
        
        # Z
        mfccZ = librosa.feature.mfcc(y=audio3, sr=20000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccZ, sr = 20000, hop_length=64)
        z_path = os.path.join(z_figpath,name+'.png')
        pylab.savefig(z_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # C
        mfccC = librosa.feature.mfcc(y=audio4, sr=20000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccC, sr = 20000, hop_length=64)
        c_path = os.path.join(c_figpath,name+'.png')
        pylab.savefig(c_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
    except:
        print('file cannot open')
        return None, None, None, None
    return x_path, y_path, z_path, c_path

In [16]:
#train_mfcc
x_figpath = '../project/data/mel_data_20k/Train_X_mfcc'
y_figpath = '../project/data/mel_data_20k/Train_Y_mfcc'
z_figpath = '../project/data/mel_data_20k/Train_Z_mfcc'
c_figpath = '../project/data/mel_data_20k/Train_C_mfcc'

x_mfcc =[]
y_mfcc =[]
z_mfcc =[]
c_mfcc =[]

for row in tqdm(trainData.values):
    x_path, y_path, z_path, c_path = feature_extractor_mfcc(row)
    x_mfcc.append(x_path)
    y_mfcc.append(y_path)
    z_mfcc.append(z_path)
    c_mfcc.append(c_path)

isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mfcc)) # check missing parts 
data_y = list(map(label,trainData['Label'])) # label: Boolean -> binary
data_x1 = [x_mfcc[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # mel-sp = x
data_x2 = [y_mfcc[i] for i in range(len(y_mfcc)) if cast_x[i] == True] # mel-sp = y
data_x3 = [z_mfcc[i] for i in range(len(z_mfcc)) if cast_x[i] == True] # mel-sp = z
data_x4 = [c_mfcc[i] for i in range(len(c_mfcc)) if cast_x[i] == True] # mel-sp = c
data_y = [data_y[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # label = y 


indices = np.arange(len(data_x1))
train_DATA_mfcc    = {}
np.random.shuffle(indices)
train_DATA_mfcc['MFCC_X'] = [data_x1[i] for i in indices]
train_DATA_mfcc['MFCC_Y'] = [data_x2[i] for i in indices]
train_DATA_mfcc['MFCC_Z'] = [data_x3[i] for i in indices]
train_DATA_mfcc['MFCC_C'] = [data_x4[i] for i in indices]
train_DATA_mfcc['LABEL'] = [data_y[i] for i in indices]


# Val mfcc 저장
x_figpath = '../project/data/mel_data_20k/Val_X_mfcc'
y_figpath = '../project/data/mel_data_20k/Val_Y_mfcc'
z_figpath = '../project/data/mel_data_20k/Val_Z_mfcc'
c_figpath = '../project/data/mel_data_20k/Val_C_mfcc'

# valData_mfcc
x_mfcc =[]
y_mfcc =[]
z_mfcc =[]
c_mfcc =[]

#저장후 이미지 경로 리스트에 저장
for row in tqdm(valData.values):
    x_path, y_path, z_path, c_path = feature_extractor_mfcc(row)
    x_mfcc.append(x_path)
    y_mfcc.append(y_path)
    z_mfcc.append(z_path)
    c_mfcc.append(c_path)

#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mfcc)) # check missing parts 
data_y = list(map(label,valData['Label'])) # label: Boolean -> binary
data_x1_val = [x_mfcc[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # mel-sp = x
data_x2_val = [y_mfcc[i] for i in range(len(y_mfcc)) if cast_x[i] == True] # mel-sp = y
data_x3_val = [z_mfcc[i] for i in range(len(z_mfcc)) if cast_x[i] == True] # mel-sp = z
data_x4_val = [c_mfcc[i] for i in range(len(c_mfcc)) if cast_x[i] == True] # mel-sp = c
data_y_val = [data_y[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # label = y 

indices = np.arange(len(data_x1_val))
test_DATA_mfcc    = {}
np.random.shuffle(indices)
test_DATA_mfcc['MFCC_X'] = [data_x1_val[i] for i in indices]
test_DATA_mfcc['MFCC_Y'] = [data_x2_val[i] for i in indices]
test_DATA_mfcc['MFCC_Z'] = [data_x3_val[i] for i in indices]
test_DATA_mfcc['MFCC_C'] = [data_x4_val[i] for i in indices]
test_DATA_mfcc['LABEL'] = [data_y_val[i] for i in indices]

pickle.dump(train_DATA_mfcc, open( "../project/data/modeling_data/V2/TrainData_XYZC_MFCC_30.pickle", "wb"))
pickle.dump(test_DATA_mfcc, open( "../project/data/modeling_data/V2/ValData_XYZC_MFCC_30.pickle", "wb"))

100%|██████████| 119/119 [00:59<00:00,  1.99it/s]


In [9]:
# # Train mel 저장
x_figpath = '../project/data/mel_data_20k/Train_X'
y_figpath = '../project/data/mel_data_20k/Train_Y'
z_figpath = '../project/data/mel_data_20k/Train_Z'
c_figpath = '../project/data/mel_data_20k/Train_C'

# valData_mel
x_mel =[]
y_mel =[]
z_mel =[]
c_mel =[]

#저장후 이미지 경로 리스트에 저장
for row in tqdm(trainData.values):
    x_path, y_path, z_path, c_path = feature_extractor(row)
    x_mel.append(x_path)
    y_mel.append(y_path)
    z_mel.append(z_path)
    c_mel.append(c_path)


#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mel)) # check missing parts 
data_y = list(map(label,trainData['Label'])) # label: Boolean -> binary
data_x1 = [x_mel[i] for i in range(len(x_mel)) if cast_x[i] == True] # mel-sp = x
data_x2 = [y_mel[i] for i in range(len(y_mel)) if cast_x[i] == True] # mel-sp = y
data_x3 = [z_mel[i] for i in range(len(z_mel)) if cast_x[i] == True] # mel-sp = z
data_x4 = [c_mel[i] for i in range(len(c_mel)) if cast_x[i] == True] # mel-sp = c
data_y = [data_y[i] for i in range(len(x_mel)) if cast_x[i] == True] # label = y 

indices = np.arange(len(data_x1))
train_DATA    = {}
np.random.shuffle(indices)
train_DATA['MEL_X'] = [data_x1[i] for i in indices]
train_DATA['MEL_Y'] = [data_x2[i] for i in indices]
train_DATA['MEL_Z'] = [data_x3[i] for i in indices]
train_DATA['MEL_C'] = [data_x4[i] for i in indices]
train_DATA['LABEL'] = [data_y[i] for i in indices]

# # Val mel 저장
x_figpath = '../project/data/mel_data_20k/Val_X'
y_figpath = '../project/data/mel_data_20k/Val_Y'
z_figpath = '../project/data/mel_data_20k/Val_Z'
c_figpath = '../project/data/mel_data_20k/Val_C'

# valData_mel
x_mel =[]
y_mel =[]
z_mel =[]
c_mel =[]

#저장후 이미지 경로 리스트에 저장
for row in tqdm(valData.values):
    x_path, y_path, z_path, c_path = feature_extractor(row)
    x_mel.append(x_path)
    y_mel.append(y_path)
    z_mel.append(z_path)
    c_mel.append(c_path)


#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mel)) # check missing parts 
data_y = list(map(label,valData['Label'])) # label: Boolean -> binary
data_x1_val = [x_mel[i] for i in range(len(x_mel)) if cast_x[i] == True] # mel-sp = x
data_x2_val = [y_mel[i] for i in range(len(y_mel)) if cast_x[i] == True] # mel-sp = y
data_x3_val = [z_mel[i] for i in range(len(z_mel)) if cast_x[i] == True] # mel-sp = z
data_x4_val = [c_mel[i] for i in range(len(c_mel)) if cast_x[i] == True] # mel-sp = c
data_y_val = [data_y[i] for i in range(len(x_mel)) if cast_x[i] == True] # label = y 

indices = np.arange(len(data_x1_val))
test_DATA    = {}
np.random.shuffle(indices)
test_DATA['MEL_X'] = [data_x1_val[i] for i in indices]
test_DATA['MEL_Y'] = [data_x2_val[i] for i in indices]
test_DATA['MEL_Z'] = [data_x3_val[i] for i in indices]
test_DATA['MEL_C'] = [data_x4_val[i] for i in indices]
test_DATA['LABEL'] = [data_y_val[i] for i in indices]

pickle.dump(train_DATA, open( "../project/data/modeling_data/V2/TrainData_XYZC_30.pickle", "wb"))
pickle.dump(test_DATA, open( "../project/data/modeling_data/V2/ValData_XYZC_30.pickle", "wb"))

100%|██████████| 80/80 [00:59<00:00,  1.34it/s]


# for RNN data Processing

In [10]:
with open("../project/data/experiment/V2/TrainData_XYZC_20.pickle", 'rb') as f:
    trainData = pickle.load(f)
with open("../project/data/experiment/V2/ValData_XYZC_20.pickle", 'rb') as f:
    valData = pickle.load(f)

In [5]:
def feature_extractor(row):      
    # Vibration
    audio = np.array(row[4])
    mfccs_per_frame = 20
    mfcc = librosa.feature.mfcc(y=audio, sr=20000, n_mfcc=mfccs_per_frame)
    return mfcc

def feature_extractor_forRNN(row, pad_length):
    name     = str(row[0])
    try:        
        # Vibration
        audio = np.array(row[4])
        mfccs_per_frame = 20
        mfcc = librosa.feature.mfcc(y=audio, sr=20000, n_mfcc=mfccs_per_frame)
        pad_width = pad_length - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width))).reshape((mfccs_per_frame,pad_length))
        
    except:
        print('file cannot open')
        return None
    return name, mfcc

In [6]:
pad_length = 0
for row in trainData.values:
    mfcc = feature_extractor(row)
    if pad_length < mfcc.shape[1] :
        pad_length = mfcc.shape[1]       
print(pad_length)

821


In [8]:
#train_mfcc
v_mfcc = []
pname = []

for row in tqdm(trainData.values):
    name, mfcc = feature_extractor_forRNN(row, pad_length)
    v_mfcc.append(mfcc)
    pname.append(name)

isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
data_y = list(map(label,trainData['Label'])) # label: Boolean -> binary
data_x = [v_mfcc[i] for i in range(len(v_mfcc))] # mel-sp = c
data_id = [pname[i] for i in range(len(pname))] # mel-sp = c
data_y = [data_y[i] for i in range(len(v_mfcc))] # label = y 

indices = np.arange(len(data_x))
train_DATA_mfcc    = {}
np.random.shuffle(indices)
train_DATA_mfcc['MFCC'] = [data_x[i] for i in indices]
train_DATA_mfcc['PNAME'] = [data_id[i] for i in indices]
train_DATA_mfcc['LABEL'] = [data_y[i] for i in indices]

# valData_mfcc
v_mfcc = []
pname = []

#저장후 이미지 경로 리스트에 저장
for row in tqdm(valData.values):
    name, mfcc = feature_extractor_forRNN(row, pad_length)
    v_mfcc.append(mfcc)
    pname.append(name)
    
#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
data_y = list(map(label,valData['Label'])) # label: Boolean -> binary
data_x_val = [v_mfcc[i] for i in range(len(v_mfcc))] # mel-sp = c
data_id_val = [pname[i] for i in range(len(pname))] # mel-sp = c
data_y_val = [data_y[i] for i in range(len(v_mfcc))] # label = y 

indices = np.arange(len(data_x_val))
test_DATA_mfcc    = {}
np.random.shuffle(indices)
test_DATA_mfcc['MFCC'] = [data_x_val[i] for i in indices]
test_DATA_mfcc['PNAME'] = [data_id_val[i] for i in indices]
test_DATA_mfcc['LABEL'] = [data_y_val[i] for i in indices]

pickle.dump(train_DATA_mfcc, open( "../project/data/modeling_data/V2/TrainData_MFCC_20_20_forRNN.pickle", "wb"))
pickle.dump(test_DATA_mfcc, open( "../project/data/modeling_data/V2/ValData_MFCC_20_20_forRNN.pickle", "wb"))

100%|██████████| 80/80 [00:02<00:00, 35.03it/s]


# for 20khz sampling rate experiment

In [2]:
with open("../project/data/experiment/V2/TrainData_XYZC_20.pickle", 'rb') as f:
    trainData = pickle.load(f)
with open("../project/data/experiment/V2/ValData_XYZC_20.pickle", 'rb') as f:
    valData = pickle.load(f)

In [6]:
def feature_extractor_SR(row):
    name     = str(row[0])
    try:  
        audio = np.array(row[4])
        
        # 8000
        melX = librosa.feature.melspectrogram(y=audio, sr=8000)
        s_db_X     = librosa.power_to_db(melX, ref=np.max)
        
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_X, sr = 8000, hop_length=64)
        x_path = os.path.join(x_figpath,name+'.png')
        pylab.savefig(x_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # 24000
        melY = librosa.feature.melspectrogram(y=audio, sr=24000)
        s_db_Y     = librosa.power_to_db(melY, ref=np.max)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_Y, sr = 24000, hop_length=64)
        y_path = os.path.join(y_figpath,name+'.png')
        pylab.savefig(y_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # 28000
        melZ = librosa.feature.melspectrogram(y=audio, sr=28000)
        s_db_Z     = librosa.power_to_db(melZ, ref=np.max)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_Z, sr = 28000, hop_length=64)
        z_path = os.path.join(z_figpath,name+'.png')
        pylab.savefig(z_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # 32000
        melC = librosa.feature.melspectrogram(y=audio, sr=32000)
        s_db_C     = librosa.power_to_db(melC, ref=np.max)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(s_db_C, sr = 32000, hop_length=64)
        c_path = os.path.join(c_figpath,name+'.png')
        pylab.savefig(c_path, bbox_inches=None, pad_inches=0)
        pylab.close()  
        
    except:
        print('file cannot open')
        return None, None, None, None
    return x_path, y_path, z_path, c_path



def feature_extractor_mfcc_SR(row):
    name     = str(row[0])
    try:  
        audio = np.array(row[4]) 
        # 16000
        mfccX = librosa.feature.mfcc(y=audio, sr=16000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccX, sr = 16000, hop_length=64)
        x_path = os.path.join(x_figpath,name+'.png')
        pylab.savefig(x_path, bbox_inches=None, pad_inches=0)
        pylab.close()     
        
        # 20000
        mfccY = librosa.feature.mfcc(y=audio, sr=20000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccY, sr = 20000, hop_length=64)
        y_path = os.path.join(y_figpath,name+'.png')
        pylab.savefig(y_path, bbox_inches=None, pad_inches=0)
        pylab.close()    
        
        # 28000
        mfccZ = librosa.feature.mfcc(y=audio, sr=28000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccZ, sr = 28000, hop_length=64)
        z_path = os.path.join(z_figpath,name+'.png')
        pylab.savefig(z_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
        # 32000
        mfccC = librosa.feature.mfcc(y=audio, sr=32000, n_mfcc=40)
        pylab.axis('off') # no axis
        pylab.axes([0., 0., 1., 1.], frameon=False, xticks=[], yticks=[])
        librosa.display.specshow(mfccC, sr = 32000, hop_length=64)
        c_path = os.path.join(c_figpath,name+'.png')
        pylab.savefig(c_path, bbox_inches=None, pad_inches=0)
        pylab.close()   
        
    except:
        print('file cannot open')
        return None, None, None, None
    return x_path, y_path, z_path, c_path

In [7]:
#train_mfcc
x_figpath = '../project/data/mel_data_20k/20kSR/Train_mfcc40_16000'
y_figpath = '../project/data/mel_data_20k/20kSR/Train_mfcc40_20000'
z_figpath = '../project/data/mel_data_20k/20kSR/Train_mfcc40_28000'
c_figpath = '../project/data/mel_data_20k/20kSR/Train_mfcc40_32000'

x_mfcc =[]
y_mfcc =[]
z_mfcc =[]
c_mfcc =[]

for row in tqdm(trainData.values):
    x_path, y_path, z_path, c_path = feature_extractor_mfcc_SR(row)
    x_mfcc.append(x_path)
    y_mfcc.append(y_path)
    z_mfcc.append(z_path)
    c_mfcc.append(c_path)

isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mfcc)) # check missing parts 
data_y = list(map(label,trainData['Label'])) # label: Boolean -> binary
data_x1 = [x_mfcc[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # mel-sp = x
data_x2 = [y_mfcc[i] for i in range(len(y_mfcc)) if cast_x[i] == True] # mel-sp = y
data_x3 = [z_mfcc[i] for i in range(len(z_mfcc)) if cast_x[i] == True] # mel-sp = z
data_x4 = [c_mfcc[i] for i in range(len(c_mfcc)) if cast_x[i] == True] # mel-sp = c
data_y = [data_y[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # label = y 


indices = np.arange(len(data_x1))
train_DATA_mfcc    = {}
np.random.shuffle(indices)
train_DATA_mfcc['MFCC_16000'] = [data_x1[i] for i in indices]
train_DATA_mfcc['MFCC_20000'] = [data_x2[i] for i in indices]
train_DATA_mfcc['MFCC_28000'] = [data_x3[i] for i in indices]
train_DATA_mfcc['MFCC_32000'] = [data_x4[i] for i in indices]
train_DATA_mfcc['LABEL'] = [data_y[i] for i in indices]


# Val mfcc 저장
x_figpath = '../project/data/mel_data_20k/20kSR/Val_mfcc40_16000'
y_figpath = '../project/data/mel_data_20k/20kSR/Val_mfcc40_20000'
z_figpath = '../project/data/mel_data_20k/20kSR/Val_mfcc40_28000'
c_figpath = '../project/data/mel_data_20k/20kSR/Val_mfcc40_32000'

# valData_mfcc
x_mfcc =[]
y_mfcc =[]
z_mfcc =[]
c_mfcc =[]

#저장후 이미지 경로 리스트에 저장
for row in tqdm(valData.values):
    x_path, y_path, z_path, c_path = feature_extractor_mfcc_SR(row)
    x_mfcc.append(x_path)
    y_mfcc.append(y_path)
    z_mfcc.append(z_path)
    c_mfcc.append(c_path)

#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mfcc)) # check missing parts 
data_y = list(map(label,valData['Label'])) # label: Boolean -> binary
data_x1_val = [x_mfcc[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # mel-sp = x
data_x2_val = [y_mfcc[i] for i in range(len(y_mfcc)) if cast_x[i] == True] # mel-sp = y
data_x3_val = [z_mfcc[i] for i in range(len(z_mfcc)) if cast_x[i] == True] # mel-sp = z
data_x4_val = [c_mfcc[i] for i in range(len(c_mfcc)) if cast_x[i] == True] # mel-sp = c
data_y_val = [data_y[i] for i in range(len(x_mfcc)) if cast_x[i] == True] # label = y 

indices = np.arange(len(data_x1_val))
test_DATA_mfcc    = {}
np.random.shuffle(indices)
test_DATA_mfcc['MFCC_16000'] = [data_x1_val[i] for i in indices]
test_DATA_mfcc['MFCC_20000'] = [data_x2_val[i] for i in indices]
test_DATA_mfcc['MFCC_28000'] = [data_x3_val[i] for i in indices]
test_DATA_mfcc['MFCC_32000'] = [data_x4_val[i] for i in indices]
test_DATA_mfcc['LABEL'] = [data_y_val[i] for i in indices]

pickle.dump(train_DATA_mfcc, open( "../project/data/modeling_data/V2/TrainData_MFCC40_SR+.pickle", "wb"))
pickle.dump(test_DATA_mfcc, open( "../project/data/modeling_data/V2/ValData_MFCC40_SR+.pickle", "wb"))

100%|██████████| 80/80 [00:43<00:00,  1.82it/s]


In [4]:
# # Train mel 저장
x_figpath = '../project/data/mel_data_20k/20kSR/Train_8000'
y_figpath = '../project/data/mel_data_20k/20kSR/Train_24000'
z_figpath = '../project/data/mel_data_20k/20kSR/Train_28000'
c_figpath = '../project/data/mel_data_20k/20kSR/Train_32000'

# valData_mel
x_mel =[]
y_mel =[]
z_mel =[]
c_mel =[]

#저장후 이미지 경로 리스트에 저장
for row in tqdm(trainData.values):
    x_path, y_path, z_path, c_path = feature_extractor_SR(row)
    x_mel.append(x_path)
    y_mel.append(y_path)
    z_mel.append(z_path)
    c_mel.append(c_path)


#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mel)) # check missing parts 
data_y = list(map(label,trainData['Label'])) # label: Boolean -> binary
data_x1 = [x_mel[i] for i in range(len(x_mel)) if cast_x[i] == True] # mel-sp = x
data_x2 = [y_mel[i] for i in range(len(y_mel)) if cast_x[i] == True] # mel-sp = y
data_x3 = [z_mel[i] for i in range(len(z_mel)) if cast_x[i] == True] # mel-sp = z
data_x4 = [c_mel[i] for i in range(len(c_mel)) if cast_x[i] == True] # mel-sp = c
data_y = [data_y[i] for i in range(len(x_mel)) if cast_x[i] == True] # label = y 

indices = np.arange(len(data_x1))
train_DATA    = {}
np.random.shuffle(indices)
train_DATA['MEL_8000'] = [data_x1[i] for i in indices]
train_DATA['MEL_24000'] = [data_x2[i] for i in indices]
train_DATA['MEL_28000'] = [data_x3[i] for i in indices]
train_DATA['MEL_32000'] = [data_x4[i] for i in indices]
train_DATA['LABEL'] = [data_y[i] for i in indices]

# # Val mel 저장
x_figpath = '../project/data/mel_data_20k/20kSR/Val_8000'
y_figpath = '../project/data/mel_data_20k/20kSR/Val_24000'
z_figpath = '../project/data/mel_data_20k/20kSR/Val_28000'
c_figpath = '../project/data/mel_data_20k/20kSR/Val_32000'

# valData_mel
x_mel =[]
y_mel =[]
z_mel =[]
c_mel =[]

#저장후 이미지 경로 리스트에 저장
for row in tqdm(valData.values):
    x_path, y_path, z_path, c_path = feature_extractor_SR(row)
    x_mel.append(x_path)
    y_mel.append(y_path)
    z_mel.append(z_path)
    c_mel.append(c_path)


#None 아닌 데이터만 추출 및 label 1,0으로 저장
isnone = lambda x: x != None
label  = lambda xx: 1 if xx == True else 0
cast_x = list(map(isnone,x_mel)) # check missing parts 
data_y = list(map(label,valData['Label'])) # label: Boolean -> binary
data_x1_val = [x_mel[i] for i in range(len(x_mel)) if cast_x[i] == True] # mel-sp = x
data_x2_val = [y_mel[i] for i in range(len(y_mel)) if cast_x[i] == True] # mel-sp = y
data_x3_val = [z_mel[i] for i in range(len(z_mel)) if cast_x[i] == True] # mel-sp = z
data_x4_val = [c_mel[i] for i in range(len(c_mel)) if cast_x[i] == True] # mel-sp = c
data_y_val = [data_y[i] for i in range(len(x_mel)) if cast_x[i] == True] # label = y 

indices = np.arange(len(data_x1_val))
test_DATA    = {}
np.random.shuffle(indices)
test_DATA['MEL_8000'] = [data_x1_val[i] for i in indices]
test_DATA['MEL_24000'] = [data_x2_val[i] for i in indices]
test_DATA['MEL_28000'] = [data_x3_val[i] for i in indices]
test_DATA['MEL_32000'] = [data_x4_val[i] for i in indices]
test_DATA['LABEL'] = [data_y_val[i] for i in indices]

pickle.dump(train_DATA, open( "../project/data/modeling_data/V2/TrainData_SR.pickle", "wb"))
pickle.dump(test_DATA, open( "../project/data/modeling_data/V2/ValData_SR.pickle", "wb"))

100%|██████████| 80/80 [01:12<00:00,  1.10it/s]
